In [1]:
import numpy as np
import pandas as pd
import torch
from datasets import load_dataset, Dataset
from utils.database import CreateDatabase, SearchDatabase
from utils.model_rag import ModelRag
from transformers import pipeline
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from sentence_transformers import SentenceTransformer
import warnings
warnings.filterwarnings('ignore')

In [2]:
np_001_15 = pd.read_excel('НП-001-15.xlsx')
np_001_15 = np_001_15.rename(columns={'НТД': 'norm', 'Пункт': 'point', 'Описание': 'description'})
dataset = Dataset.from_pandas(np_001_15)

In [3]:
dataset[0]

{'document': 'НП-001-15',
 'point': '1.1.1',
 'decription': 'Большой аварийный выброс - выброс радиоактивных веществ в окружающую среду при аварии на АС, при котором необходимо выполнение мер защиты населения на границе зоны планирования защитных мероприятий на начальном периоде аварии, установленной в соответствии с требованиями норм и правил по размещению АС, и за ее пределами.'}

In [4]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [5]:
generation_pipeline = pipeline(
    'text-generation',
    model='Qwen/Qwen2-1.5B-Instruct',
    device=device,
    torch_dtype=torch.float16
)

Sliding Window Attention is enabled but not implemented for `sdpa`; unexpected results may be encountered.
Device set to use cuda


In [6]:
embedding_model = SentenceTransformer("intfloat/multilingual-e5-large", model_kwargs={'torch_dtype': torch.float16})

In [7]:
cross_encoder = HuggingFaceCrossEncoder(
    model_name='amberoad/bert-multilingual-passage-reranking-msmarco',
    model_kwargs={'device': device}
)

In [8]:
# create_db = CreateDatabase(
#     embedding_model=embedding_model,
#     dataset=dataset,
#     device=device,
#     collection_name='regulatory_documentation',
#     path='aep_chat_bot'    
# )

# create_db.create()

In [9]:
model = ModelRag(
    generation_pipeline=generation_pipeline,
    embedding_model=embedding_model,
    cross_encoder=cross_encoder,
    path_client='aep_chat_bot',
    device=device,
    collection_name='regulatory_documentation',
    temperature = 0.9,
    top_p = 0.6,
    max_new_tokens=1024
)

In [18]:
query = 'Опиши четыре класса безопасности по НП-001-15. Расскажи подробно что они включают и какие эелементы к ним относятся? В каком пункте НП-001-15 идет их описание?'
answer, context = model.predict(query)
print(answer)

В соответствии с вашим запросом, здесь приведен список классов безопасности по НП-001-15:

1. Класс 1. Элементы АС, которые могут быть исходными событиями аварий, приводящими к повреждению тепловыделяющего элемента и превышению максимального проектного предела.

2. Класс 2. Элементы АС, которые не входят в класс 1, но могут быть исходными событиями аварий, приводящими к повреждению тепловыделяющего элемента без превышения максимального проектного предела при проектном функционировании систем безопасности с учетом нормируемого для проектных аварий количества отказов в этих системах.

3. Класс 3. Элементы АС, которые важны для безопасности и не входят в классы 1 и 2.

4. Класс 4. Элементы АС, которые не влияют на безопасность и не вошли в классы 1, 2, 3. 

Они содержатся в пункте 7, разделе "Основные понятия" статьи NPP-001-15.


In [11]:
query = 'Расскажи про 5 уровней глубокоэшелонированной защиты'
answer, context = model.predict(query)
print(answer)

Ваш вопрос о 5 уровнях глубокоэшелонированной защиты в атомной энергетике имеет множество различных источников, включая официальные нормативные документы, руководства по проектированию и правила безопасности. Вот несколько основных аспектов, которые могут быть связаны с вашим вопросом:

1. Задачи уровня 1: Оценка и выбор площадки, пригодной для размещения АС; установление санитарно-защитной зоны, зоны наблюдения вокруг АС, а также зоны планирования защитных мероприятий. Этот уровень может включать в себя анализ местоположения, доступности и возможностей защиты, а также проведение исследований для определения наиболее безопасного места для размещения АС.

2. Задачи уровня 2: Предотвращение проектных аварий системами нормальной эксплуатации: своевременное выявление отклонений от нормальной эксплуатации и их устранение; управление при эксплуатации с отклонениями. Этот уровень может включать в себя регулярные проверки и диагностику системы, а также внедрение систем контроля качества для пр

In [12]:
query = 'Перечисли основные функции систем безопасности на атомной станции'
answer, context = model.predict(query)
print(answer)

Основные функции систем безопасности на атомной станции включают:

1. Аварийный останов реактора и поддержание его в подкритическом состоянии: система обеспечивает своевременное прекращение работы реактора при возникновении аварии и защиту окружающей среды от радиации.

2. Аварийный отвод тепла от реактора: система обеспечивает эффективное отведение тепла от реактора и бассейнов выдержки отработавшего ядерного топлива, предотвращая их нагрев и дальнейшее радиоактивное загрязнение.

3. Удержание радиоактивных веществ в установленных границах: система обеспечивает постоянное контроль за уровнем радиоактивности ядерного топлива и других радиоактивных веществ, чтобы предотвратить их неблагоприятное воздействие на окружающую среду и здоровье людей.

4. Функции защиты: система обеспечивает защиту оборудования и людей от воздействия радиации и других опасностей.

5. Бассейны для выдержки ОЯТ: система обеспечивает подпитку бассейнов для хранения и транспортировки ядерного топлива, чтобы предот